# LASSO/RIDGE Logistic Regression Model
- source: R & H

In [64]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score
import csv

def read_csv(filename):
    return pd.read_csv(filename).to_dict('records')

WB_database_codes = read_csv('WB_categories.csv')
data_categories = []
i = 0
for element in WB_database_codes:
    data_categories.append(WB_database_codes[i]['Code'])
    i += 1

In [54]:
# Load the dataset
df_MENA = pd.read_csv("merged_dfs_MENA_noNA.csv")
df_MENA = df_MENA.drop(columns=["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0_x", "code", "geo_x", "economy", "Unnamed: 0_y", "country_x", "Time", "geo_y", "country_y", "Unnamed: 0", "iso3"])

# Removal of removed data categories
removed_data_categories = ['SI.POV.MDIM', 'SI.POV.MDIM.XQ', 'SI.POV.MDIM.IT', 'SH.MED.CMHW.P3', 'SH.MED.NUMW.P3', 'SL.WAG.0714.ZS']
for cat in removed_data_categories:
    data_categories.remove(cat)

df_MENA["time_period"] = ( df_MENA["year"] - 2004 )*12 + df_MENA["month"] - 7 # convert date into an integer
df_MENA = df_MENA[["date", "country_code", "Country", "year", "month", "time_period", "protest", "revolution", "riots", "strike", "unrest", "violence"] + data_categories + ["unrest_index"]]
df_MENA
df_MENA.to_csv(r'merged_dfs_MENA_noNA_v2.csv')

In [48]:
# Load the dataset - PENDING ANDRES update
df_world = pd.read_csv("merged_dfs_world_noNA.csv")
df_world = df_world.reset_index()
df_world = df_world.drop(columns=["Unnamed: 0.1", "Unnamed: 0_x", "code", "geo_x", "economy", "Unnamed: 0_y", "country_x", "Time", "geo_y", "country_y", "Unnamed: 0", "iso3"])

# Removal of removed data categories
removed_data_categories = ['SI.POV.MDIM', 'SI.POV.MDIM.XQ', 'SI.POV.MDIM.IT', 'SH.MED.CMHW.P3', 'SH.MED.NUMW.P3', 'SL.WAG.0714.ZS']
for cat in removed_data_categories:
    data_categories.remove(cat)

df_MENA["time_period"] = ( df_MENA["year"] - 2004 )*12 + df_MENA["month"] - 7 # convert date into an integer
df_MENA = df_MENA[["date", "country_code", "Country", "year", "month", "time_period", "protest", "revolution", "riots", "strike", "unrest", "violence"] + data_categories + ["unrest_index"]]
df_MENA

KeyError: "['code', 'geo_x', 'country_x', 'geo_y', 'country_y'] not found in axis"

In [65]:
# Set the dependent variable and independent variables
y = df_MENA['unrest_index']
X = df_MENA.drop(['date', 'unrest_index', 'country_code', 'Country', 'year', 'month'], axis=1)

# Set the number of years to predict and train on
num_months = len(df_MENA['time_period'].unique())
train_months = 36
num_months

# Split the data into training and testing sets based on years
tscv = TimeSeriesSplit(n_splits=num_months - train_months)

# Initialize the Lasso model
lasso = Lasso(alpha=0.1)

In [63]:

# Iterate through each year and fit the model
for train_index, test_index in tscv.split(X):
    # Set the training and testing data for the current year
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Select only the past three years for training data
    X_train = X_train[X_train['time_period'] >= X_test['time_period'].min() - train_months]

    # Fit the Lasso model on the training data
    lasso.fit(X_train, y_train)

    # Predict the probability of the dependent variable for the current year
    y_pred = lasso.predict(X_test)

    # Print the mean squared error for the current year
    print('Month:', X_test['time_period'].min(), 'MSE:', ((y_pred - y_test) ** 2).mean())


/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.572e+01, tolerance: 8.632e-03
  model = cd_fast.enet_coordinate_descent(
/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.681e+01, tolerance: 8.917e-03
  model = cd_fast.enet_coordinate_descent(
/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

Month: 143 MSE: 1.4791980540886536
Month: 149 MSE: 0.10202333357645792
Month: 155 MSE: 0.20151692389340534
Month: 161 MSE: 0.11639537816946266
Month: 167 MSE: 0.14967589112687396
Month: 173 MSE: 0.009694141666013581
Month: 138 MSE: 248.04737910867223
Month: 142 MSE: 0.06548665856360902
Month: 148 MSE: 0.06114846253603904
Month: 154 MSE: 0.0342352346849499
Month: 160 MSE: 0.07019516221450593
Month: 166 MSE: 1.6651813915878055
Month: 172 MSE: 0.06013676414521522
Month: 138 MSE: 508.58292160790796
Month: 141 MSE: 0.2292449541563044
Month: 147 MSE: 0.022830262272704658
Month: 153 MSE: 0.02970516153518407
Month: 159 MSE: 0.037118862132953546
Month: 165 MSE: 0.10483978771174286
Month: 171 MSE: 0.14614940981271382
Month: 138 MSE: 37.1612167515385
Month: 140 MSE: 0.017089401374794467


/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.481e+01, tolerance: 1.178e-02
  model = cd_fast.enet_coordinate_descent(
/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.494e+01, tolerance: 1.185e-02
  model = cd_fast.enet_coordinate_descent(
/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

Month: 146 MSE: 0.0011189414991161517
Month: 152 MSE: 0.0009974622636199008
Month: 158 MSE: 0.0754495154654097
Month: 164 MSE: 0.03128003458322909
Month: 170 MSE: 0.10960031221521649
Month: 0 MSE: 0.10748923843340479
Month: 1 MSE: 0.016038386927095034
Month: 7 MSE: 0.011900916244744432
Month: 13 MSE: 0.01069959928715918
Month: 19 MSE: 0.038717655968606816
Month: 25 MSE: 0.04732347829656588
Month: 31 MSE: 0.0244615227960012


/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.616e+01, tolerance: 1.210e-02
  model = cd_fast.enet_coordinate_descent(
/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.626e+01, tolerance: 1.219e-02
  model = cd_fast.enet_coordinate_descent(
/Users/matthewchiu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

ValueError: Found input variables with inconsistent numbers of samples: [246, 247]

In [68]:
# loop over each country and perform LASSO regression
for country in df_MENA['Country'].unique():
    # filter the data for the current country
    X_country = X[df_MENA['Country']==country]
    y_country = y[df_MENA['Country']==country]
    
    # initialize an empty list to store the predicted probabilities
    prob_list = []
    
    # loop over the time series splits and perform LASSO regression
    for train_index, test_index in tscv.split(X_country):
        # split the data into training and testing sets
        X_train = X_country.iloc[train_index]
        y_train = y_country.iloc[train_index]
        X_test = X_country.iloc[test_index]
        y_test = y_country.iloc[test_index]
        
        # perform LASSO regression
        lasso = Lasso(alpha=0.1)
        lasso.fit(X_train, y_train)
        
        # predict the probability of conflict for the test set
        prob = lasso.predict(X_test)
        
        # add the predicted probabilities to the list
        prob_list.extend(prob)
    
    # add the predicted probabilities to the original dataset
    df_MENA.loc[df_MENA['Country']==country, 'prob_conflict'] = prob_list
    
    # calculate the AUC score for the current country
    auc_score = roc_auc_score(y_country, df_MENA.loc[df_MENA['Country']==country, 'prob_conflict'])
    print(f"AUC score for {country}: {auc_score}")

ValueError: Cannot have number of folds=146 greater than the number of samples=38.